In [21]:
import math

first_frame = 159 # first frame that the blower leaves
last_frame = 617 # last frame just before blower hits wall

height_body = 1.295 # height of blower in meters
radius_body = 0.94 / 2 # radius of blower in meters

height_top = 0.147 # height of top of blower in meters
radius_top = 0.474 / 2 # radius of top of blower in meters

total_time = (last_frame - first_frame) / 30 # total time in seconds
speed = 43 / 2.237 # max speed of animal converted from mph to m/s

In [22]:
volume_body = math.pi * radius_body**2 * height_body # volume of blower in m^3
volume_air = math.pi * (radius_body - 0.04)**2 * (height_body - 0.04) # volume of air in m^3

volume_top = math.pi * radius_top**2 * height_top # volume of top of blower in m^3

volume_steel = volume_body - volume_air + volume_top # volume of steel in m^3

density_aluminium = 2700 # density of aluminium in kg/m^3
mass_aluminium = volume_steel * density_aluminium # mass of aluminium in kg

density_air = 1.293 # density of air in kg/m^3
mass_air = volume_air * density_air # mass of air in kg

In [23]:
# weerstand (grond), versnelling (lucht)

weight = mass_aluminium + mass_air # weight of blower in kg
normal_force = 9.81 * weight

roll_friction_coefficient = 0.01 # friction coefficient of the ground with the wheels

ground_friction_newton = roll_friction_coefficient * normal_force

frontal_area = 2 * math.pi * radius_body * height_body / 2 # frontal area of blower in m^2
air_friction_newton = 0.5 * density_air * speed**2 * 0.47 * frontal_area # air friction in newton

total_friction = ground_friction_newton + air_friction_newton # total friction in newton

214.67863694457253
